# Import thư viện

### Nạp thư viện

In [ ]:
!python -m pip install --upgrade pip

! pip -q install --upgrade pip
! pip -q install -r req.txt
! pip install -U accelerate
! pip install -U transformers

### Import thư viện

In [43]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder

# Tiền xử lí dữ liệu



### Nạp model

In [55]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base-v2", num_labels=6)

c:\Users\qscvd\OneDrive\Documents\GitHub\makerthon\phoBERT-finetuning\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\qscvd\.cache\huggingface\hub\models--vinai--phobert-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of RobertaForSequenceClassification were not initialized fr

#### lưu model (optional)

In [57]:
model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\bpe.codes',
 'tokenizer\\added_tokens.json')

### Nạp dataset

In [74]:
# Tải dữ liệu từ file CSV
dataset = load_dataset("csv", data_files={"train": "dataset.csv", "test": "dataset.csv"})

# Lấy danh sách các nhãn gốc từ cột 'intent'
intent_labels = dataset["train"]["intent"]  # Cột 'intent' chứa các nhãn gốc

# Khởi tạo LabelEncoder để mã hóa nhãn thành chỉ số
label_encoder = LabelEncoder()

# Mã hóa các nhãn gốc thành các chỉ số
label_encoder.fit(intent_labels)  # Fit trên toàn bộ nhãn từ cột 'intent'

# In ra các nhãn gốc và các chỉ số đã mã hóa
print("Danh sách nhãn gốc:", label_encoder.classes_)  # In mảng nhãn gốc
print("Các nhãn đã mã hóa:", label_encoder.transform(intent_labels))  # In nhãn đã mã hóa thành chỉ số

# Mã hóa nhãn intent thành chỉ số và tokenize văn bản
def encode_labels_and_tokenize(examples):
    # Tokenize văn bản
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=13)
    
    # Mã hóa intent thành nhãn (labels) sử dụng LabelEncoder đã được fit
    tokenized["labels"] = label_encoder.transform(examples["intent"])  # Mã hóa nhãn thành chỉ số
    
    return tokenized

# Áp dụng mã hóa nhãn và tokenization cho cả train và test dataset
train_dataset = dataset["train"].map(encode_labels_and_tokenize, batched=True)
test_dataset = dataset["test"].map(encode_labels_and_tokenize, batched=True)

# Kiểm tra dữ liệu sau khi mã hóa nhãn và tokenization
  
print(train_dataset["labels"])  # Hiển thị mẫu đầu tiên trong train dataset

Generating train split: 521 examples [00:00, 81706.20 examples/s]
Generating test split: 521 examples [00:00, 113494.98 examples/s]


Danh sách nhãn gốc: ['clear_history' 'find_api' 'find_history' 'find_news' 'log_in' 'log_out']
Các nhãn đã mã hóa: [2 1 5 4 5 4 1 1 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5

Map: 100%|██████████| 521/521 [00:00<00:00, 8327.71 examples/s]

[2, 1, 5, 4, 5, 4, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

# Finetuning

### Cấu hình tham số huấn luyện

In [76]:
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir='./logs',
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=10  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


### Finetuning

In [ ]:
trainer.train()

model.save_pretrained("./results/final_model")
tokenizer.save_pretrained("./results/final_model")

c:\Users\qscvd\OneDrive\Documents\GitHub\makerthon\phoBERT-finetuning\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.688600
20,0.438900
30,0.304000


# Test output

In [62]:
tokenizer = AutoTokenizer.from_pretrained("./results/final_model")
model = AutoModelForSequenceClassification.from_pretrained("./results/final_model", num_labels=6)

In [68]:
import torch

# Ví dụ văn bản muốn dự đoán
text = "Tôi muốn đọc các bài viết về sự phát triển của thị trường bất động sản"

# Tokenize văn bản
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Dự đoán với mô hình
with torch.no_grad():
    outputs = model(**inputs)

# Lấy kết quả dự đoán (logits)
logits = outputs.logits

# Lấy nhãn dự đoán (với argmax)
predicted_label = logits.argmax(dim=-1).item()

print(f"Predicted label: {predicted_label}")


Predicted label: 3
